# 🏦 Banking-RAG Chatbot Projesi

## Proje Tanıtımı

Bu projede, **Banking77** veri setini kullanarak, banka müşterilerinin sorularına akıllıca cevap verebilen bir **RAG (Retrieval-Augmented Generation)** tabanlı chatbot geliştireceğiz.

### 🎯 RAG Mimarisi Nedir?

RAG, bilgi getirme (retrieval) ve üretken dil modeli (generation) teknolojilerini birleştiren bir mimaridir:

1. **Retrieval (Bilgi Getirme):** Kullanıcının sorusuyla ilgili en alakalı dokümanlar vektör veritabanından getirilir
2. **Augmentation (Zenginleştirme):** Getirilen bilgiler prompt içine eklenir
3. **Generation (Üretim):** LLM bu bilgileri kullanarak doğru ve tutarlı yanıt üretir

### 🛠️ Kullanılacak Teknolojiler

- **LLM (Generative Model):** Gemini API (gemini-2.5-flash)
- **Embedding Model:** sentence-transformers/all-mpnet-base-v2
- **Vektör Veritabanı:** ChromaDB
- **RAG Framework:** LangChain
- **Veri Seti:** PolyAI/banking77 (Hugging Face)
- **Web Arayüzü:** Streamlit

### 📊 Veri Seti Hakkında

Banking77 veri seti, 13.083 müşteri sorusunu içeren ve 77 farklı niyeti (intent) kapsayan bir veri setidir. Bankacılık alanında yaygın müşteri sorgularını içerir.

## 📦 Gerekli Kütüphanelerin Kurulumu

Bu bölümde projemiz için gerekli tüm Python kütüphanelerini yüklüyoruz.

In [ ]:
# Gerekli kütüphaneleri yükle

# LangChain
%pip install -q langchain==0.1.16 langchain-community==0.0.36 langchain-google-genai==0.0.8

# Vector DB
%pip install -q chromadb==0.4.24

# Embeddings / ML
%pip install -q sentence-transformers==2.7.0 transformers==4.41.2

# PyTorch
%pip install -q torch==2.2.2 

# Google AI
%pip install -q google-generativeai==0.3.2

# Data Processing
%pip install -q pandas==2.2.2 numpy==1.26.4

print("✅ Tüm kütüphaneler başarıyla yüklendi!")

## 📚 Kütüphaneleri İçe Aktar

Projede kullanacağımız tüm kütüphaneleri içe aktarıyoruz.

In [ ]:
# Temel kütüphaneler
import os
import warnings
warnings.filterwarnings('ignore')

# Veri işleme
import pandas as pd
import numpy as np

# Embedding
from sentence_transformers import SentenceTransformer

# ChromaDB
import chromadb

# LangChain
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Gemini API
import google.generativeai as genai

print("✅ Tüm kütüphaneler başarıyla içe aktarıldı!")

## 🔑 API Anahtarını Ayarla

Gemini API'yi kullanabilmek için API anahtarınızı buraya girmeniz gerekiyor.

**Not:** API anahtarınızı [Google AI Studio](https://makersuite.google.com/app/apikey) adresinden alabilirsiniz.

In [ ]:
# Gemini API anahtarını buraya girin
# API anahtarınızı https://makersuite.google.com/app/apikey adresinden alabilirsiniz
GOOGLE_API_KEY = "YOUR_API_KEY_HERE"  # Buraya kendi API anahtarınızı girin

# Ortam değişkenine ata
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Gemini API'yi yapılandır
genai.configure(api_key=GOOGLE_API_KEY)

print("✅ API anahtarı başarıyla ayarlandı!")

## 📊 Veri Seti Yükleme ve İnceleme

Banking77 veri setini Hugging Face'den yüklüyoruz ve içeriğini inceliyoruz.

In [ ]:
# Banking77 veri setini CSV olarak yükle (sadece pandas kullan)
print("⏳ Veri seti yükleniyor...")

try:
    # Kaynak CSV'ler
    train_url = "https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv"
    test_url = "https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv"

    # CSV'leri yükle
    train_df = pd.read_csv(train_url)
    test_df = pd.read_csv(test_url)

    # Kolon isimlerini düzelt (category -> label) ve string'e çevir
    if 'category' in train_df.columns:
        train_df = train_df.rename(columns={'category': 'label'})
    if 'category' in test_df.columns:
        test_df = test_df.rename(columns={'category': 'label'})

    # Label'ların string olduğundan emin ol
    train_df['label'] = train_df['label'].astype(str)
    test_df['label'] = test_df['label'].astype(str)

    # Basit özet
    print("✅ Veri seti başarıyla yüklendi!")
    print(f"📊 Train set: {len(train_df)} örnek")
    print(f"📊 Test set: {len(test_df)} örnek")
    print(f"📋 Kolonlar: {list(train_df.columns)}")
    print(f"🏷️ Toplam intent sayısı: {train_df['label'].nunique()}")

except Exception as e:
    print(f"❌ Veri seti yüklenirken hata oluştu: {e}")
    raise

In [ ]:
# İlk 10 örneği incele
print("\n📖 İlk 10 örnek:")
print(train_df.head(10))

# İstatistiksel bilgiler
print(f"\n📈 Benzersiz intent (niyet) sayısı: {train_df['label'].nunique()}")
print(f"\n🏷️ İlk 10 intent dağılımı:")
print(train_df['label'].value_counts().head(10))

In [ ]:
# Örnek sorguları göster
print("\n💬 Örnek Müşteri Sorguları:")
print("=" * 80)
for i, text in enumerate(train_df['text'].head(10), 1):
    print(f"{i}. {text}")
print("=" * 80)

## 🧠 Embedding Modelinin Yüklenmesi ve Kullanımı

Sentence Transformers kütüphanesinden **all-mpnet-base-v2** modelini yüklüyoruz. Bu model, metinleri 768 boyutlu vektörlere dönüştürür.

In [ ]:
# Embedding modelini yükle
print("⏳ Embedding modeli yükleniyor...")
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
print("✅ Embedding modeli başarıyla yüklendi!")

# Model hakkında bilgi
print(f"\n📐 Embedding boyutu: {embedding_model.get_sentence_embedding_dimension()}")

In [ ]:
# Örnek metin için embedding oluştur
ornek_metin = "I lost my credit card, what should I do?"
ornek_embedding = embedding_model.encode(ornek_metin)

print(f"📝 Örnek Metin: {ornek_metin}")
print(f"\n📊 Embedding Vektör Boyutu: {ornek_embedding.shape}")
print(f"\n🔢 İlk 10 embedding değeri:")
print(ornek_embedding[:10])
print(f"\n📈 Vektör istatistikleri:")
print(f"   - Minimum: {ornek_embedding.min():.4f}")
print(f"   - Maksimum: {ornek_embedding.max():.4f}")
print(f"   - Ortalama: {ornek_embedding.mean():.4f}")

In [ ]:
# Birkaç farklı sorgu için embedding oluştur ve benzerlik hesapla (numpy ile)

def cosine_sim_np(a: np.ndarray, b: np.ndarray) -> float:
    a = a.astype(np.float32)
    b = b.astype(np.float32)
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

test_sorulari = [
    "I lost my credit card",
    "How can I transfer money?",
    "What is my account balance?"
]

test_embeddings = embedding_model.encode(test_sorulari)

print("\n🔍 Test Sorguları ve Embedding Boyutları:")
for i, (soru, emb) in enumerate(zip(test_sorulari, test_embeddings), 1):
    print(f"{i}. '{soru}' -> Boyut: {emb.shape}")

# İlk soru ile diğerleri arasındaki benzerlik
print("\n📏 Cosine Benzerlik Skorları (1. soru ile karşılaştırma):")
base = test_embeddings[0]
for i, soru in enumerate(test_sorulari[1:], 2):
    benzerlik = cosine_sim_np(base, test_embeddings[i-1])
    print(f"   Soru {i}: {benzerlik:.4f}")

## 💾 ChromaDB Veritabanı Kurulumu

Tüm Banking77 veri setindeki metinleri embedding'lere dönüştürüp ChromaDB'ye kaydediyoruz.

In [ ]:
# Tüm metinler için embedding oluştur
print("⏳ Tüm veri seti için embedding'ler oluşturuluyor...")
print(f"📊 Toplam işlenecek metin sayısı: {len(train_df)}")

# Metinleri al
all_texts = train_df['text'].tolist()
all_labels = train_df['label'].astype(str).tolist()

# Batch halinde embedding oluştur (daha hızlı)
batch_size = 32
all_embeddings = embedding_model.encode(
    all_texts,
    batch_size=batch_size,
    show_progress_bar=True
)

# NumPy -> list
all_embeddings = np.asarray(all_embeddings)

print(f"\n✅ {len(all_embeddings)} adet embedding başarıyla oluşturuldu!")
print(f"📐 Embedding matrisi boyutu: {all_embeddings.shape}")

In [ ]:
# ChromaDB client oluştur
print("⏳ ChromaDB başlatılıyor...")

# Streamlit uygulaması ile aynı yol ve isimleri kullan
chroma_db_path = "./chroma_db"
collection_name = "banking77_collection"

# Persistent Client
chroma_client = chromadb.PersistentClient(path=chroma_db_path)

# Varsa eski koleksiyonu sil (opsiyonel güvenli)
try:
    chroma_client.delete_collection(name=collection_name)
    print(f"🗑️ Eski '{collection_name}' koleksiyonu silindi.")
except Exception:
    pass

# Yeni koleksiyon oluştur
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"description": "Banking77 dataset embeddings"}
)

print(f"✅ '{collection_name}' koleksiyonu oluşturuldu!")

In [ ]:
# Embedding'leri ChromaDB'ye ekle
print("⏳ Embedding'ler ChromaDB'ye ekleniyor...")

batch_size = 100
n = len(all_texts)
for i in range(0, n, batch_size):
    batch_end = min(i + batch_size, n)
    collection.add(
        embeddings=all_embeddings[i:batch_end].tolist(),
        documents=all_texts[i:batch_end],
        ids=[f"id_{j}" for j in range(i, batch_end)],
        metadatas=[{"label": all_labels[j]} for j in range(i, batch_end)]
    )
    if (i // batch_size) % 10 == 0:
        print(f"   ⏳ {batch_end}/{n} embedding eklendi...")

print(f"\n✅ Toplam {n} embedding başarıyla ChromaDB'ye eklendi!")
print(f"📊 Koleksiyon bilgileri:")
print(f"   - İsim: {collection.name}")
print(f"   - Toplam kayıt: {collection.count()}")

## 🔍 ChromaDB Sorgulama Testi

Basit bir sorgulama örneği ile sistemin çalışıp çalışmadığını test ediyoruz.

In [ ]:
# Test sorgusu
test_sorgu = "I lost my card"

# Sorgu için embedding oluştur
sorgu_embedding = embedding_model.encode([test_sorgu])

# En yakın 5 sonucu getir
sonuclar = collection.query(
    query_embeddings=sorgu_embedding.tolist(),
    n_results=5
)

print(f"🔍 Test Sorgusu: '{test_sorgu}'")
print(f"\n📋 En yakın {len(sonuclar['documents'][0])} sonuç:\n")
print("=" * 80)

for i, (doc, distance) in enumerate(zip(sonuclar['documents'][0], sonuclar['distances'][0]), 1):
    print(f"\n{i}. Sonuç:")
    print(f"   📝 Metin: {doc}")
    print(f"   📏 Uzaklık Skoru: {distance:.4f}")

print("\n" + "=" * 80)

## 🧩 RAG Pipeline Kurulumu

LangChain kullanarak tam bir RAG pipeline'ı oluşturuyoruz. Bu pipeline:
1. Kullanıcı sorgusunu alır
2. ChromaDB'den en alakalı dokümanları getirir
3. Bu bilgileri Gemini API'ye gönderir
4. Anlamlı ve doğru yanıt üretir

In [ ]:
# LangChain için embedding fonksiyonu oluştur
print("⏳ LangChain bileşenleri hazırlanıyor...")

# HuggingFace embeddings wrapper
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# ChromaDB'yi LangChain vector store olarak yapılandır
vectorstore = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embeddings
)

print("✅ Vector store hazır!")

In [ ]:
# Gemini LLM modelini yapılandır
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    google_api_key=GOOGLE_API_KEY,
)

print("✅ Gemini LLM modeli hazır!")

In [ ]:
# Prompt şablonu oluştur
prompt_template = """
You are a helpful banking assistant. Use the context below to answer the question accurately and professionally.

If the question is related to the provided context, give a detailed and helpful answer.
If the question is not related to banking or the context, politely say that you can only help with banking-related questions.

Context:
{context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("✅ Prompt şablonu oluşturuldu!")

In [ ]:
# RAG chain'i oluştur
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
)

print("✅ RAG Pipeline başarıyla oluşturuldu!")
print("\n📋 Pipeline özellikleri:")
print("   - Retrieval sayısı: 3 en alakalı doküman")
print("   - LLM modeli: gemini-2.5-flash")
print("   - Embedding modeli: all-mpnet-base-v2")

## 🧪 RAG Pipeline Test

Oluşturduğumuz RAG sistemini test ediyoruz.

In [ ]:
# Test fonksiyonu
def test_rag_system(question):
    """
    RAG sistemini test etmek için yardımcı fonksiyon
    """
    print(f"\n{'='*80}")
    print(f"❓ SORU: {question}")
    print(f"{'='*80}")
    
    # RAG chain'den yanıt al
    result = rag_chain({"query": question})
    
    # Yanıtı göster
    print(f"\n💡 CEVAP:\n{result['result']}")
    
    # Kaynak dokümanları göster
    print(f"\n📚 KAYNAK DOKÜMANLAR:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\n   {i}. {doc.page_content}")
    
    print(f"\n{'='*80}\n")
    
    return result

print("✅ Test fonksiyonu hazır!")

In [ ]:
# Test sorusu 1
test_rag_system("I lost my credit card, what should I do?")

In [ ]:
# Test sorusu 2
test_rag_system("How can I transfer money to another account?")

In [ ]:
# Test sorusu 3
test_rag_system("What are the fees for international transactions?")

## 📊 Performans Değerlendirme

RAG sisteminin performansını değerlendirmek için bazı metrikler hesaplıyoruz.

In [ ]:
import time

# Performans testi
test_questions = [
    "I lost my card",
    "How to transfer money?",
    "What is my balance?",
    "How to activate card?",
    "ATM withdrawal limit"
]

print("🧪 Performans testi başlatılıyor...\n")
print("=" * 80)

total_time = 0
results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 Test {i}/{len(test_questions)}: {question}")
    
    # Zaman ölç
    start_time = time.time()
    result = rag_chain({"query": question})
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    total_time += elapsed_time
    
    print(f"   ⏱️ Yanıt süresi: {elapsed_time:.2f} saniye")
    print(f"   📄 Kaynak doküman sayısı: {len(result['source_documents'])}")
    
    results.append({
        'question': question,
        'time': elapsed_time,
        'sources': len(result['source_documents'])
    })

print("\n" + "=" * 80)
print("\n📈 PERFORMANS ÖZETİ:")
print(f"   ⏱️ Toplam süre: {total_time:.2f} saniye")
print(f"   📊 Ortalama yanıt süresi: {total_time/len(test_questions):.2f} saniye")
print(f"   ✅ Başarıyla tamamlanan test: {len(results)}/{len(test_questions)}")

## 🎯 Sonuç ve Öneriler

### ✅ Proje Başarıyla Tamamlandı!

Bu notebook'ta şunları gerçekleştirdik:

1. ✅ **Veri Yükleme:** Banking77 veri seti Hugging Face'den yüklendi
2. ✅ **Embedding Oluşturma:** 13,000+ metin için vektör temsilleri oluşturuldu
3. ✅ **Vektör Veritabanı:** ChromaDB ile semantik arama altyapısı kuruldu
4. ✅ **RAG Pipeline:** LangChain ve Gemini API ile tam bir RAG sistemi oluşturuldu
5. ✅ **Web Arayüzü:** Streamlit ile kullanıcı dostu bir chatbot arayüzü geliştirildi

### 🚀 Geliştirme Önerileri:

1. **Çok Dilli Destek:** Türkçe sorguları da destekleyecek şekilde genişletilebilir
2. **Fine-tuning:** Gemini modeli bankacılık domaininde fine-tune edilebilir
3. **Kullanıcı Geri Bildirimi:** Yanıt kalitesi için rating sistemi eklenebilir
4. **Önbellek Mekanizması:** Sık sorulan sorular için cache eklenebilir
5. **Analytics Dashboard:** Kullanım istatistikleri ve metrikler için dashboard oluşturulabilir

### 📚 Kaynaklar:

- [Banking77 Dataset](https://huggingface.co/datasets/PolyAI/banking77)
- [LangChain Documentation](https://python.langchain.com/)
- [Gemini API](https://ai.google.dev/)
- [ChromaDB](https://www.trychroma.com/)
- [Sentence Transformers](https://www.sbert.net/)

## 📦 ChromaDB'yi ZIP'e Dönüştür ve İndir

Aşağıdaki hücre, `./chroma_db` klasörünü `chroma_db.zip` olarak arşivler. Böylece GitHub'a kolayca ekleyebilirsiniz.

In [ ]:
# chroma_db klasörünü zip'le
import shutil
import os
from pathlib import Path

chroma_db_path = Path("./chroma_db").resolve()
zip_output = Path("./chroma_db.zip").resolve()

if not chroma_db_path.exists():
    print("❌ 'chroma_db' klasörü bulunamadı. Önce embedding oluşturma hücrelerini çalıştırın.")
else:
    # Varsa eski zip'i sil
    if zip_output.exists():
        zip_output.unlink()
    
    # Windows uyumlu arşivleme
    archive_base = str(zip_output.with_suffix(''))  # ".../chroma_db"
    shutil.make_archive(archive_base, 'zip', root_dir=str(chroma_db_path))

    size_mb = os.path.getsize(zip_output) / (1024 * 1024)
    print(f"✅ Arşiv oluşturuldu: {zip_output}")
    print(f"📦 Boyut: {size_mb:.2f} MB")
    print("ℹ️ Bu dosyayı GitHub deposuna ekleyebilirsiniz: 'chroma_db.zip'")